In [ ]:
# ENVIRONMENT

from aisurveywriter.core.llm_handler import LLMHandler
import aisurveywriter.core.file_handler as fh
from aisurveywriter.utils import get_all_files_from_paths
from aisurveywriter.core.pipeline import PaperPipeline
from aisurveywriter.core.paper import PaperData
import aisurveywriter.tasks as tks

import os
os.environ["GOOGLE_API_KEY"]=fh.read_credentials("../credentials.yaml")["google_key"]

# llm = LLMHandler(model="qwen2.5:14b", model_type="ollama", temperature=0.5)
prompts = fh.read_yaml("../templates/prompt_config.yaml")
review = fh.read_yaml("../templates/review_config.yaml")

In [ ]:
from aisurveywriter.store.prompt_store import PromptStore, default_prompt_store
import json

old = default_prompt_store()

with open("prompts-24022025.json", "w", encoding="utf-8") as f:
    json.dump(old.model_dump(), f, indent=2)

In [ ]:
generate_struct = r"""- You are an expert in academic writing, specially in the field of **{subject}**.

1. **GOAL**: generate a detailed structure for a scientific survey paper on the subject "{subject}". Be formal, comprehensive and organize the subject logically and thoroughly.
- Base this structure on the provided references in reference_content block.

2. **INSTRUCTIONS**:
- Do not include sections **Abstract** or **References**;
- Include core  sections/subsections and any additional ones inspired by the provided sources and your analysis of the subject.
- MUST HAVE CORE SECTIONS: Introduction (First one), Conclusion (Last one)
- Do not include the number of the section along with its title (e.g. 1. introduction, 2. methods...)
- Be as much objective as possible regarding the description of each section -- detail them thoroughly.
- Do not include sections that may be repetitive
- Minimum of 8 sections.

- Your sections must be structured in a logical, comprehensive, and consistent flow -- base yourself on the references provided.
- This structure is for a SURVEY paper. So make sure your sections cover the fundamentals of the entire content thoroughly.

3. **OUTPUT FORMAT**:
- Provide the structure in JSON format. THE TILE MUST BE ENCLOSED WITH QUOTES
- Do it as follows:
    ```json
    {{
        "sections": [
        {{ 
            "title": "Section1 Title",
            "description": "- Subsection Title\n\t- Explanation of what this subsection will cover.\n\t- Additional details as necessary.\n- Another Subsection Title\n\t- Explanation.\n\t- More details."
        }},
        {{
            "title": "Section2 title",
            "descripton": "- Subsection title\n- More explanation..."
        }}
        ]
    }}
    ``` 

**Provide your answer in English only**."""

write_section = r"""
**Follow these strict guidelines for LaTeX academic writing:**
- Format: Use only numbered sectioning (\section, \subsection, etc.). Exclude the LaTeX preamble.
- Citations: Do not include citations or bibliography commands.
- Figures/Tables: If a section is under 500 words, include at least one visual element:
    - Use tables where relevant.
    - For visual explanations, use TikZ (without \includegraphics).
- Writing Style:
    - Write in English with a formal, scientific, and objective tone.
    - At least 500 words per section—the more detailed, the better.
    - Avoid numbered/unnumbererd lists and redundancy; write fluid, structured text.

**Task**:
- You are an expert academic writer in {subject}, creating a comprehensive survey based on provided references.
- Incorporate direct insights from these works and their cited sources, but do not copy.
- Cover fundamental topics, ensuring depth and completeness.
- Write section by section as the user provides titles and descriptions.
- You may add subsections, but no other sectioning.

**Strictly adhere to:**
The section title and description, provided by the human.
The provided references.

Output only LaTeX content
"""

apply_section = r"""- **Role:** You are an expert academic writer in **{subject}**.  
- **Input:** You will receive a LaTeX section of a survey paper, review directives, and reference content from PDFs.  
- **Task:** Apply the review directives while maintaining the section’s integrity and improving its quality based on references.  

### **Guidelines**  
- **Maintain style:** Keep a **scientific, objective, and formal tone**.  
- **Preserve structure:** Do **not** add or remove sections (\section) or subsections (\subsection).  
- **Expand, don’t summarize:** Maintain or increase length with meaningful content.  
- **Discuss figures:** Refer to existing figures via their **\label** and \caption details. Do **not** replace standard figures with TikZ.  
- **Visual elements:**  
  - Use **TikZ** for explanatory illustrations (except for existing figures).  
  - Use **tabular** elements where categorization aids clarity.  

### **Formatting Rules**  
- **English only**.  
- **LaTeX only** (no preamble, no bibliography commands).  
- **Do not cite (\cite, \bibliography, etc.).**  
- **Output only the revised LaTeX content**—no additional comments or explanations.  

**Apply changes strictly as directed. If no change is required, output the section as-is.**
"""

old_write_section = r"""**BASE YOUR KNOWLEDGE IN THE REFERENCES PROVIDED BY THE HUMAN; !ALL YOUR FOLLOWING OUTPUT MUST ADHERE TO!:**

**You must use only LaTeX format, using the following guidelines for structure and content:**
- LaTeX Sectioning:
- Use only numbered sectioning commands (\section, \subsection, etc.) (don't use unnumbered such as \section*).
- Exclude the LaTeX preamble (e.g., \documentclass, \usepackage, \begin{{document}} and \end{{document}}).

- Citations and References:
- DO NOT try to add citations anywhere. Focus only in writing.
- DO NOT use any kind of BibTex or bibliography-related commands (\printbibliography, \cite, \bibliography, etc)

- Figures and Tables:
- If the section is too short (less than 500 words), you must include at least one visual element:
    - Include tables that are adequate for the explanation of a topic
    - If there is some visual topic to be explained, USE TIKZ TO CREATE A VISUALIZATION
    - Tickz pictures MUST NOT include extern pictures (with \includegraphics)

- **Write in English only**.

- **You are an expert in academic writing, specially in the field of {subject}.**

- Your task is to write a complete survey on {subject} based on the references that are given.

- You are pragmatic, so you are writing it section by section, as the user provides you with a section title and its description.

- From your experience, you know that you must incorporate direct insights from the authors, from works cited by these authors, and insights you gather yourself.

- As an academic writer, **you must inspire yourself with as many references as possible**. Nevertheless, do not copy the references.

- Be sure to cover the most fundamental topics for the section. These topics can easily be recognized based on their appearance on the references.

- Avoid using bullet points. If you need to detail and/or explain a topic, write it within the flow of the text.

- Avoid any repetition of contents.

- The more you adhere to this guidelines, the better your performance -- always increase your performance.

- You can add subsections, but no other section.

- **Strictly** follow the current section Title and Description, and base your content on the provided references.

- Your output must be only the latex content, nothing else.

- Most important: maintain a formal, scientific, and objective tone. **THE MORE YOU WRITE, THE BETTER. YOU SHOULD WRITE AT LEAST 500 WORDS IN A SECTIION**."""

old_apply = r"""- **You are an expert in academic writing, specially in the field of "{subject}".**

- You will receive from the human: references contents (from PDFs), content of one section for a survey paper in LaTeX, and instructions from a review of the paper.

- You must apply the review directives, taking into account the context provided by the references.

## Requirements

- **Maintain the same style of scientific, objective, and formal writing.**

- **Maintain or increase the length by incorporating meaningful content.**

- **DO NOT SUMMARIZE ANY STEP, ONLY DETAIL IT.**

- **Preserve formatting**

- **Preserve all sections (\section) and subsections (\subsection). You should not remove nor add any.**

- Avoid bullet points for detailing/explaining topics. Add them to the flow of the text.

## Output format
- **Provide all your answers in English**.

- **Use only LaTeX** (no preamble commands neither biblatex commands)

- YOUR OUTPUT SHOULD CONTAIN NOTHING MORE THAN JUST THE LATEX OUTPUT FOR THIS SECTION, as:

\"\"\"
\section{{(*section title*)}}
...
\subsection{{...}}
...
\"\"\"

- DO NOT USE ANY BIBLATEX COMMAND ANYWHERE (including \cite, \bibliography, \printbibliography, \begin{{thebibliography}}, etc)
- DO NOT ADD REFERENCES, ONLY FOLLOW STRICTLY THE REVIEW DIRECTIVES (DONT USE \cite ANYWHERE)

- INCLUDE VISUAL ELEMENTS WITH Tickz PACKAGE OR TABULAR ELEMENTS WHERE RELEVANT:
    - It is relevant to include figures (USE TIKZ) where a topic can be easily explained visually. Create a visual explanation or demonstration of the topic in this case
    - It is relevant to include tabular elements where a topic can be categorized and explained better in that way
    - Do not replace standard figures with TikZ.
- Standard figures (added using \begin{{figure}}..\includegraphics...\end{{figure}}) MUST NOT BE REPLACED BY TIKZ. 

- Remember to discuss about figures using their "\label" and getting information about them from "\caption"

- FOCUS ONLY IN THIS SECTION. You can add subsections, but no other section.

- **Strictly** follow the current section Title and Description and its Review Directives, and base your content on the provided references.

- If there is no clear directive to make a change, then don't change anything and output the section as-it-is.

- **TASK**: Follow these directives strictly and apply improvements to the given section. Do not remove nor add any sections."""
